### Cargar librerias

In [1]:
pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 kB 27.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#Guardar modelos
from scipy.sparse import csr_matrix
from scipy import sparse


#LDA
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from pprint import pprint

In [3]:
from google.colab import drive
drive.flush_and_unmount()


Drive not mounted, so nothing to flush and unmount.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
drive.mount('/content/drive')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [5]:
# # Leer el archivo CSV desde github: datos de cluster  asignado a cada usuario

# df_clusters = pd.read_csv('/content/drive/My Drive/df_clusters.csv')


# # Mostrar las primeras filas del DataFrame
# print(df_clusters.head())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Cargar datos

In [6]:
file_path_cluster = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/df_clusters.csv"

# df_clusters = pd.read_csv(file_path_cluster)

df_clusters = pd.read_csv('/content/drive/My Drive/df_clusters.csv')

print(df_clusters.head(1))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   Unnamed: 0  userId  cluster
0           0  123687        4


In [7]:
file_path_df_final = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/df_final.csv"

#df_final = pd.read_csv(file_path_df_final)

df_final = pd.read_csv('/content/drive/My Drive/df_final.csv')

print(df_final.head(1))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   userId  movieId  rating   id  original_title  cluster
0  123687      163     4.0  163  Ocean's Twelve        4


In [8]:
file_path_meta = "https://raw.githubusercontent.com/juankquintana/aprendizaje-no-supervisado/main/Proyecto/data/movies_metadata.csv"

metadata = pd.read_csv(file_path_meta)

metadata.head(1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-8-8c32b58a7750>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(file_path_meta)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [9]:
file_path_usuarios = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/base_usuarios.csv"

df_usuarios = pd.read_csv(file_path_usuarios)

df_usuarios = df_usuarios.drop(df_usuarios.columns[0], axis=1)

df_usuarios.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating
0,123687,163,4.0
1,203854,163,5.0
2,78563,163,1.0
3,203732,163,4.0
4,195546,163,5.0


In [10]:
#Descargar achivo desde github a entorno de ejecución local

df_keywords_title = pd.read_pickle('ModelosNLP/df_keywords_title.pkl')
df_keywords_title.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,original_title,keyword_name,cleaned_keywords,genre_names
0,2,Ariel,"[underdog, prison, factory worker, prisoner, h...","[Drama, Crime, underdog, prison, factori worke...","[Drama, Crime]"
1,3,Varjoja paratiisissa,"[salesclerk, helsinki, garbage, independent film]","[Drama, Comedy, salesclerk, helsinki, garbag, ...","[Drama, Comedy]"
2,5,Four Rooms,"[hotel, new year's eve, witch, bet, hotel room...","[Crime, Comedy, hotel, new year eve, witch, be...","[Crime, Comedy]"
3,6,Judgment Night,"[chicago, drug dealer, boxing match, escape, o...","[Action, Thriller, Crime, chicago, drug dealer...","[Action, Thriller, Crime]"
4,11,Star Wars,"[android, galaxy, hermit, death star, lightsab...","[Adventure, Action, Science Fiction, android, ...","[Adventure, Action, Science Fiction]"


In [43]:
#Cargar datos para recomendacion por genero

file_path_user_gener = "https://media.githubusercontent.com/media/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/user_gener_c.csv"

user_gener_c = pd.read_csv(file_path_user_gener)

#user_gener_c=pd.read_csv('/content/user_gener_c.csv')
user_gener_c = user_gener_c.dropna(subset=['genre'])
user_gener_c.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,timestamp,original_title,genre_names,genre_names_exploded,genre
0,123687,163,4.0,1259948252,Ocean's Twelve,"['Thriller', 'Crime']",'Thriller',Thriller
1,123687,163,4.0,1259948252,Ocean's Twelve,"['Thriller', 'Crime']",'Crime',Crime
2,221195,4973,4.0,1465065341,Sous le Sable,"['Drama', 'Mystery']",'Drama',Drama
3,221195,4973,4.0,1465065341,Sous le Sable,"['Drama', 'Mystery']",'Mystery',Mystery
4,134366,1643,4.0,960702344,Ultimo tango a Parigi,"['Drama', 'Romance']",'Drama',Drama


Creamos un nuevo data frame integrando usuarios con el nombre de la película

In [13]:
df_usuarios['movieId']=df_usuarios['movieId'].astype(str)

df_merge = pd.merge(df_usuarios, df_keywords_title, left_on='movieId', right_on='id', how='inner')

# Seleccionamos las columnas necesarias: userId, movieId, original_title y rating.
df_ratings_peliculas = df_merge[['userId', 'movieId', 'original_title', 'rating']]

# Mostramos el dataframe resultante.
print(df_ratings_peliculas.head(3))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   userId movieId  original_title  rating
0  123687     163  Ocean's Twelve     4.0
1  203854     163  Ocean's Twelve     5.0
2   78563     163  Ocean's Twelve     1.0


In [14]:
df_ratings_peliculas.dtypes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
userId,int64
movieId,object
original_title,object
rating,float64


### Cargamos la función para correr las recomendaciones utilizando clustering y similitud de los usuarios intra cluser (similitud del coseno): esta función no será usada finalmente en el recomendador final

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

# Creamos función recomendadora

def recomendacion_cluster(user_id):
    # Asegurarse de que el user_id sea tratado como string (si es necesario)
    user_id = str(user_id)
    df_clusters['userId'] = df_clusters['userId'].astype(str)
    df_final['userId'] = df_final['userId'].astype(str)

    # Obtener cluster asignado al usuario
    cluster = df_clusters.loc[df_clusters['userId'] == user_id, 'cluster'].values[0]
    print("El cluster del usuario es: ",cluster)

    # Filtrar df_final por el cluster
    df_filtered = df_final[df_final['cluster'] == cluster]
    print("La forma del data set filtrado por este cluster es: ",df_filtered.shape)
    # verificar usuarios y peliculas
    num_unique_users = df_filtered['userId'].nunique()
    num_unique_movies = df_filtered['movieId'].nunique()
    print("El número de usuarios y peliculas es : ",num_unique_users,", ",num_unique_movies)

    # Obtener las películas que ya ha visto el usuario
    peliculas_vistas = df_filtered[df_filtered['userId'] == user_id]['original_title'].unique()

    # Crear la matriz de relaciones entre usuarios, películas y ratings
    r_matrix = df_filtered.pivot_table(values='rating', index='userId', columns='original_title', fill_value=0)
    print("La forma de la matriz del data set filtrado por este cluster es: ",r_matrix.shape)

    # Verificar si el usuario está en la matriz
    if user_id not in r_matrix.index:
        print(f"El usuario {user_id} no se encuentra en la matriz de calificaciones para este clúster.")
        return pd.DataFrame()  # Devolver un DataFrame vacío si el usuario no existe


    # Calcular la similitud del coseno entre los usuarios
    cosine_sim = cosine_similarity(r_matrix)
    cosine_sim_df = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

    # Obtener las similitudes del usuario actual con otros
    sim_scores = cosine_sim_df[user_id]

    # Crear lista para almacenar los promedios ponderados
    peliculas_wmeans = []

    # Iterar sobre cada columna (película) de la matriz de reputación
    for pelicula in r_matrix.columns:
        # Verificar si la película ya fue vista por el usuario
        if pelicula in peliculas_vistas:
            continue  # Saltar a la siguiente película si ya fue vista

        # Obtener los ratings de la película
        p_rating = r_matrix[pelicula]

        # Filtrar usuarios que no han calificado la película
        mask_rated = p_rating > 0
        sim_scores_filtered = sim_scores[mask_rated]
        p_rating_filtered = p_rating[mask_rated]

        # Calcular la media ponderada de los ratings
        if len(p_rating_filtered) > 0:
            wmean_rate = np.dot(sim_scores_filtered, p_rating_filtered) / sim_scores_filtered.sum()
            #print("Pelicula: ",pelicula," wmean_rate, ",wmean_rate)
        else:
            wmean_rate = 3.0  # Valor por defecto si no hay suficientes datos

        peliculas_wmeans.append((pelicula, wmean_rate))

    # Crear DataFrame con los resultados y ordenar
    df_wmeans = pd.DataFrame(peliculas_wmeans, columns=['original_title', 'wmean_rating'])
    df_wmeans_sorted = df_wmeans.sort_values(by='wmean_rating', ascending=False)

    return df_wmeans_sorted.head(10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Testeamos función

In [16]:
recomendacion_cluster(123687)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


El cluster del usuario es:  4
La forma del data set filtrado por este cluster es:  (1101786, 6)
El número de usuarios y peliculas es :  30679 ,  4881
La forma de la matriz del data set filtrado por este cluster es:  (30679, 4813)


<ipython-input-15-660dc6b98a8d>:62: RuntimeWarning: invalid value encountered in scalar divide
  wmean_rate = np.dot(sim_scores_filtered, p_rating_filtered) / sim_scores_filtered.sum()


,original_title,wmean_rating
1021,Dou Sing 2 - Gai Tau Dou Sing,5.0
3138,Skazka Stranstviy,5.0
3280,Sudden Danger,5.0
2880,Red-Headed Woman,5.0
2674,Paranoia,5.0
779,Copycat,5.0
95,A Kid for Two Farthings,5.0
1485,Harvard Man,5.0
4425,White Zombie,5.0
1494,Hell's Highway,5.0


### Implementamos SVD con 500 valores singulares por la limitación de espacio y memoria para correrlo

In [17]:
import psutil
mem = psutil.virtual_memory()
print(f'Memoria total: {mem.total / (1024**3):.2f} GB')
print(f'Memoria disponible: {mem.available / (1024**3):.2f} GB')

Memoria total: 334.56 GB
Memoria disponible: 328.21 GB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
##Utilice este espacio para escribir el código.
#Codigo original del taller
A_df = df_ratings_peliculas.pivot_table(index='userId', columns='movieId',values='rating').fillna(0)
A_df.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


movieId,100,100010,100017,100042,100046,100089,100167,100183,100196,100246,...,993,99343,99351,994,996,99642,99826,99846,99861,99946
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
A = A_df.to_numpy()
user_count_ponderado_mean = np.mean(A, axis = 1)
A_demeaned = A - user_count_ponderado_mean.reshape(-1, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# Supongamos que A es tu matriz densa
A_demeaned_df = pd.DataFrame(A_demeaned)

# Mostrar las primeras filas
print(A_demeaned_df.head())

       0         1         2         3         4         5         6     \
0 -0.000804 -0.000804 -0.000804 -0.000804 -0.000804 -0.000804 -0.000804   
1 -0.002211 -0.002211 -0.002211 -0.002211 -0.002211 -0.002211 -0.002211   
2 -0.000603 -0.000603 -0.000603 -0.000603 -0.000603 -0.000603 -0.000603   
3 -0.002814 -0.002814 -0.002814 -0.002814 -0.002814 -0.002814 -0.002814   
4 -0.002412 -0.002412 -0.002412 -0.002412 -0.002412 -0.002412 -0.002412   

       7         8         9     ...      4965      4966      4967      4968  \
0 -0.000804 -0.000804 -0.000804  ... -0.000804 -0.000804 -0.000804 -0.000804   
1 -0.002211 -0.002211 -0.002211  ... -0.002211 -0.002211 -0.002211 -0.002211   
2 -0.000603 -0.000603 -0.000603  ... -0.000603 -0.000603 -0.000603 -0.000603   
3 -0.002814 -0.002814 -0.002814  ... -0.002814 -0.002814 -0.002814 -0.002814   
4 -0.002412 -0.002412 -0.002412  ... -0.002412 -0.002412 -0.002412 -0.002412   

       4969      4970      4971      4972      4973      4974  
0 -0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# Convertirlo en un DataFrame para facilitar la visualización
user_count_ponderado_mean_df = pd.DataFrame(user_count_ponderado_mean, columns=['mean_rating'])

# Mostrar las primeras filas
print(user_count_ponderado_mean_df.head())

   mean_rating
0     0.000804
1     0.002211
2     0.000603
3     0.002814
4     0.002412


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(A_demeaned, k = 500)
sigma = np.diag(sigma)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
print("U shape:", U.shape)
print("Sigma shape:", sigma.shape)
print("Vt shape:", Vt.shape)

U shape: (226890, 500)
Sigma shape: (500, 500)
Vt shape: (500, 4975)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
# Transformacion en matriz identidad con sigmas en sy diagonal

A_recomendaciones = np.dot(np.dot(U, sigma), Vt)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# se añade la media que se habia restado incialmente
A_recomendaciones = np.dot(np.dot(U, sigma), Vt) + user_count_ponderado_mean.reshape(-1, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
A_recomendaciones_df = pd.DataFrame(A_recomendaciones, index=A_df.index, columns=A_df.columns)
A_recomendaciones_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


movieId,100,100010,100017,100042,100046,100089,100167,100183,100196,100246,...,993,99343,99351,994,996,99642,99826,99846,99861,99946
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.002357,-0.000199,-0.000123,-0.000233,-0.000143,0.000758,-0.000046,-0.000004,-0.000024,0.000008,...,0.000414,0.000017,0.000146,-0.000582,0.002883,0.000666,-0.000118,-0.000226,0.000169,0.000007
2,0.052256,-0.000882,-0.000807,-0.000012,-0.000709,0.000860,-0.000740,-0.000490,-0.001608,0.000460,...,0.007463,-0.000204,0.000006,-0.002251,-0.001118,-0.001224,-0.000925,-0.001087,0.000327,-0.000676
3,0.017239,-0.000278,-0.000246,0.000216,-0.000472,-0.000112,-0.000253,-0.000128,-0.000755,-0.000149,...,0.000312,-0.000094,0.000210,-0.002253,0.012193,-0.000053,-0.000383,-0.000373,-0.000488,0.001736
4,0.026807,0.000482,-0.000451,0.002676,-0.000527,-0.000598,-0.000360,-0.000425,-0.000873,-0.000237,...,0.002422,-0.000371,-0.000747,-0.006446,0.024273,-0.000118,-0.000628,-0.000979,-0.000514,-0.000440
5,0.004289,0.000995,0.001002,0.001093,0.000849,0.000838,0.001036,0.001071,0.000542,0.000961,...,0.001244,0.001036,0.000878,0.044227,-0.004605,0.001335,0.000937,0.001238,0.000784,0.000974


## Funciones auxiliares

In [27]:
### Peliculas ya vistas por el ususario
def rated_movies_by_user(user_id, df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]

    # Verificar si el usuario tiene películas registradas
    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)

    # Retornar todos los movieId, título y rating como una lista de tuplas
    return list(zip(user_movies_sorted['movieId'], user_movies_sorted['original_title'], user_movies_sorted['rating']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
###Top 3 peliculas del usuario

def top_movies_by_user(user_id, top_n=3,df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]

    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)

    # Seleccionar el top N de películas
    top_movies = user_movies_sorted.head(top_n)

    # Retornar el movieId y el rating como una lista de tuplas
    return list(zip(top_movies['movieId'], top_movies['original_title'],top_movies['rating']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
###Top 5 peliculas del usuario

def top_movies_by_user5(user_id, top_n=5,df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]

    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)

    # Seleccionar el top N de películas
    top_movies = user_movies_sorted.head(top_n)

    # Asegurarse de que 'indice' y 'id' tengan el mismo tipo de dato
    top_movies.loc[:,'movieId'] = top_movies['movieId'].astype(str)
    df_keywords_title.loc[:,'id'] = df_keywords_title['id'].astype(str)

    # Unir con el DataFrame de recomendaciones usando el índice de 'final_recommendations'
    top_movies = pd.merge(top_movies,
                      df_keywords_title[['id', 'genre_names']].rename(columns={'genre_names': 'Generos'}),
                      left_on='movieId',
                      right_on='id',
                      how='inner')

    # Eliminar la columna 'id' de top_movies
    top_movies = top_movies.drop(columns=['id'])

    # Crear un nuevo DataFrame con las columnas que quieres mostrar
    df_resultado = top_movies.loc[:,['movieId', 'original_title', 'rating', 'Generos']]

    # Imprimir el DataFrame
    return df_resultado


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



## Generamos un recomendador con SVD

In [29]:
def recomendar_peliculas_svd(user_id):
    # Obtener las recomendaciones para el usuario dado
    recomendaciones_svd = A_recomendaciones_df.loc[user_id]

    # Obtener las películas que el usuario ya ha visto desde df_ratings_peliculas
    peliculas_usuario = df_ratings_peliculas[df_ratings_peliculas['userId'] == user_id]['movieId']

    # Filtrar películas que el usuario ya ha visto
    recomendaciones_svd_filtradas = recomendaciones_svd[~recomendaciones_svd.index.isin(peliculas_usuario)]

    # Ordenar las recomendaciones y seleccionar las mejores
    top_svd = recomendaciones_svd_filtradas.sort_values(ascending=False).reset_index()
    top_svd.columns = ['movieId', 'value']

    # Asegurarse de que 'movieId' y 'id' tengan el mismo tipo de dato
    df_keywords_title['id'] = df_keywords_title['id'].astype(str)  # Ajusta el tipo de dato si es necesario
    top_svd['movieId'] = top_svd['movieId'].astype(str)

    # Unir con el DataFrame de metadata para obtener los nombres de las películas
    top_svd = pd.merge(top_svd, df_keywords_title[['id', 'original_title','genre_names']], left_on='movieId', right_on='id', how='left')

    # Renombrar columnas y eliminar la columna 'id'
    top_svd = top_svd.rename(columns={
        'original_title': 'Titulo',
        'value': 'valor svd',
        'movieId': 'Indice',
        'genre_names': 'Genero'
    }).drop(columns=['id'])


    # Retornar el top 5 de recomendaciones con nombre de película
    return top_svd.head(5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Testeamos función

In [30]:
recomendar_peliculas_svd(123687)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Indice,valor svd,Titulo,Genero
0,2528,1.164571,Madame Bovary,"[Drama, Foreign, Romance]"
1,2009,0.694973,"4 luni, 3 săptămîni și 2 zile",[Drama]
2,8914,0.585808,Deep Blue Sea,"[Action, Science Fiction, Thriller]"
3,1831,0.563071,The Next Best Thing,[Comedy]
4,8983,0.448371,Felidae,"[Animation, Mystery, Thriller]"


## Generamos función recomendadora utilizando NPL: Natural Language Processing

In [31]:
#Toca descargar directo
from scipy import sparse
cosine_sim_sparseCOUNT = sparse.load_npz('/content/cosine_sim_sparse_matrixCOUNT.npz')

# Convertirla de nuevo aU densa (si es necesario, aunque puedes seguir usándola como dispersa)
cosine_sim_sparseCOUNT = cosine_sim_sparseCOUNT.toarray()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
cosine_sim_sparseCOUNT[8605]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.21821789, 0.15430335, 0.        , ..., 0.        , 0.        ,
       0.18257419])

In [33]:
def recomendador_multiple(movies_list, cosine_sim=cosine_sim_sparseCOUNT, num_recomendaciones=5, df=df_keywords_title):
    # Lista para almacenar los DataFrames de recomendaciones
    all_recommendations = []

    for movie in movies_list:
        movie_id = movie[0]  # ID de la película
        # Crear una Serie que contiene como índice el 'id' y el valor correspondiente al índice de la película
        indices = pd.Series(df.index, index=df['id'])

        # Verificar si el movie_id está en los índices
        if movie_id not in indices.index:
            print(f'ID de película {movie_id} no encontrado.')
            continue

        # Obtener el índice de la película a partir del movie_id
        idx = indices[movie_id]

        # Obtener el título de la película original
        original_title = df.loc[idx, 'original_title']
        #print(f'Título de la película original: {original_title}')
        #print(f'Índice de la película: {movie_id}')

        # Obtener la similitud de coseno para esa película en particular con todas las películas
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordenar la lista de tuplas por la similitud de coseno en orden descendente
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtener las películas más similares ignorando el primero (que es la misma película)
        sim_scores = sim_scores[1:num_recomendaciones + 1]

        # Obtener los índices y las similitudes
        movie_indices = [i[0] for i in sim_scores]
        sim_scores_values = [i[1] for i in sim_scores]

        # Verificar que los índices obtenidos están dentro del rango del DataFrame
        valid_indices = [idx for idx in movie_indices if idx in df.index]

        valid_id = [df['id'].iloc[idx] for idx in valid_indices]

        # Crear un DataFrame para mostrar los resultados
        result_df = pd.DataFrame({
            'IdPelicula': valid_id,
            'Título': df['original_title'].iloc[valid_indices].values,
            'Similitud Coseno': [sim_scores_values[movie_indices.index(idx)] for idx in valid_indices]
        })


        # Añadir el DataFrame de recomendaciones a la lista
        all_recommendations.append(result_df)

    # Concatenar todos los DataFrames en uno solo
    final_recommendations = pd.concat(all_recommendations, ignore_index=True)

    final_recommendations.sort_values(by='Similitud Coseno', ascending=False, inplace=True)

    final_recommendations.reset_index(drop=True,inplace=True)


    # Asegurarse de que 'indice' y 'id' tengan el mismo tipo de dato
    final_recommendations['IdPelicula'] = final_recommendations['IdPelicula'].astype(str)
    df_keywords_title['id'] = df_keywords_title['id'].astype(str)

    # Unir con el DataFrame de recomendaciones usando el índice de 'final_recommendations'
    final_recommendations = pd.merge(final_recommendations,
                                 df_keywords_title[['id', 'genre_names']].rename(columns={'genre_names': 'Generos'}),
                                 left_on='IdPelicula',  # Usamos el índice de final_recommendations
                                 right_on='id',    # Y la columna 'id' de df_keywords_title
                                 how='inner')


    # Imprimir las películas recomendadas
    #print('\nPelículas recomendadas:')
    #print(final_recommendations)

    # Retornar el DataFrame con los resultados
    return final_recommendations.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Testeamos función

In [34]:
lista = top_movies_by_user(123687, top_n=3,df=df_ratings_peliculas)
lista

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('2288', 'Closer', 5.0),
 ('1967', 'Ask the Dust', 5.0),
 ('1372', 'Blood Diamond', 4.5)]

In [35]:
recomendador_multiple(lista, cosine_sim=cosine_sim_sparseCOUNT, num_recomendaciones=5, df=df_keywords_title)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,IdPelicula,Título,Similitud Coseno,id,Generos
0,5123,August Rush,0.617161,5123,"[Family, Drama, Music]"
1,4369,Juste une question d'amour,0.606339,4369,"[Drama, Romance]"
2,235984,Goliyon Ki Raasleela Ram-Leela,0.593366,235984,"[Drama, Romance]"
3,6003,Romeo and Juliet,0.585540,6003,"[Drama, Romance]"
4,27904,แสงศตวรรษ,0.577350,27904,"[Drama, Romance]"


## Recomendador por género en base a promedios simple y ponderado

In [37]:
def recomendar_peliculas_por_genero():
    # Pedir al usuario que ingrese un género de película
    genero_usuario = input(
        "Por favor, ingrese un género de película que le interese. Las opciones son:\n"
        "'Thriller', 'Crime', 'Drama', 'Mystery', 'Romance', 'Comedy',\n"
        "'Science Fiction', 'Adventure', 'Family', 'Horror', 'History',\n"
        "'TV Movie', 'Fantasy', 'Action', 'Foreign', 'Music',\n"
        "'Documentary', 'Animation', 'Western', 'War'\n"
    ).lower()

    # Filtrar el DataFrame por el género ingresado, aplicando lower para evitar problemas
    df_filtrado = user_gener_c[user_gener_c['genre'].str.lower().str.contains(genero_usuario)]

    # Calcular métricas globales de las películas dentro del género seleccionado
    df_peliculas = df_filtrado.groupby(['movieId', 'original_title']).agg(
        usuarios_unicos=('userId', 'nunique'),   # Contar valores únicos de 'userId'
        rating_pelicula=('rating', 'sum')        # Sumar el rating total
    ).reset_index()

    total_usuarios = df_peliculas['usuarios_unicos'].sum()
    total_rating = df_peliculas['rating_pelicula'].sum()

    # Calcular las probabilidades sin ponderar y ponderada
    df_peliculas['prob_sin_ponderar'] = (df_peliculas['usuarios_unicos'] / total_usuarios)
    df_peliculas['prob_ponderada'] = (df_peliculas['rating_pelicula'] / total_rating)

    # Ordenar películas por cantidad de usuarios (sin ponderar)
    sorted_peliculas_sin_ponderar = df_peliculas.sort_values(by='prob_sin_ponderar', ascending=False)

    # Tomar las primeras 10 películas sin ponderar
    recomendaciones_sin_ponderar = sorted_peliculas_sin_ponderar.head(10)[['original_title', 'prob_sin_ponderar']]

    # Ordenar películas por rating (ponderado)
    sorted_peliculas_ponderada = df_peliculas.sort_values(by='prob_ponderada', ascending=False)

    # Tomar las primeras 10 películas ponderadas
    recomendaciones_ponderada = sorted_peliculas_ponderada.head(10)[['original_title', 'prob_ponderada']]

    # Concatenar ambas listas de recomendaciones
    df_recomendaciones = pd.concat([recomendaciones_ponderada[['original_title']], recomendaciones_sin_ponderar[['original_title']]])

    # Eliminar duplicados de películas
    df_recomendaciones = df_recomendaciones.drop_duplicates(subset='original_title').reset_index(drop=True)

    # Mostrar el mensaje con las recomendaciones
    print(f"Estas son las películas recomendadas en el género '{genero_usuario}', ¡dale una oportunidad!\n")
    return df_recomendaciones.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Recomendador final que integra las 3 funciones recomendadoras elegidas

In [40]:
def recomendacion_personalizada(df_ratings_peliculas=df_ratings_peliculas, df_keywords_title=df_keywords_title, cosine_sim_sparseCOUNT=cosine_sim_sparseCOUNT):
    # Solicitar el ID del usuario
    try:
        user_id = int(input("Por favor, introduce tu ID de usuario: "))
    except ValueError:
        print("ID de usuario no válido. Se recomendarán películas por género.")
        return recomendar_peliculas_por_genero()

    print(f"ID de usuario ingresado: {user_id}")

    # Obtener las películas mejor valoradas por el usuario
    try:
        top_movies = top_movies_by_user(user_id)
        if not top_movies:
            print(f'No hay películas disponibles para el usuario {user_id}. Se recomendarán películas por género.')
            return recomendar_peliculas_por_genero()
    except Exception as e:
        print(f"Error al obtener películas para el usuario {user_id}: {e}. Se recomendarán películas por género.")
        return recomendar_peliculas_por_genero()

    # Obtener las recomendaciones basadas en las mejores películas con NLP
    recomendaciones_NLP = recomendador_multiple(top_movies, cosine_sim=cosine_sim_sparseCOUNT, df=df_keywords_title)

    # Obtener las películas ya vistas por el usuario
    try:
        vistas = rated_movies_by_user(user_id)
        vistas_ids = {v[0] for v in vistas}  # Obtener solo los movieId de las películas vistas
    except Exception as e:
        print(f"Error al obtener películas vistas por el usuario {user_id}: {e}")
        vistas_ids = set()  # Si hay error, tratamos como si no hubiera películas vistas

    # Filtrar las recomendaciones para eliminar las películas vistas
    recomendaciones_filtradas_NLP = recomendaciones_NLP[~recomendaciones_NLP['IdPelicula'].isin(vistas_ids)]

    # Obtener las recomendaciones basadas en SVD
    recomendaciones_SVD = recomendar_peliculas_svd(user_id)

    # Imprimir las recomendaciones finales
    print(f'\nUsuario {user_id}. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores (NLP):')
    if recomendaciones_filtradas_NLP.empty:
        print("No hay nuevas recomendaciones disponibles.")
    elif recomendaciones_SVD.empty:
      print("No hay nuevas recomendaciones disponibles.")
    else:
      return (recomendaciones_filtradas_NLP,recomendaciones_SVD)


    # # Imprimir las recomendaciones finales de SVD
    # print(f'\nAquí tienes unas recomendaciones de películas que aún no has visto según las preferencias de usuarios similares a ti (SVD):')
    # if recomendaciones_SVD.empty:
    #     print("No hay nuevas recomendaciones disponibles.")
    # else:
    #     print(recomendaciones_SVD)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Análisis resultados usuarios



Análisis de usuario nuevo

In [ ]:
# Asumiendo que no conocieramos el usuario y elegimos drama
recomendar_peliculas_por_genero()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Por favor, ingrese un género de película que le interese. Las opciones son:
'Thriller', 'Crime', 'Drama', 'Mystery', 'Romance', 'Comedy',
'Science Fiction', 'Adventure', 'Family', 'Horror', 'History',
'TV Movie', 'Fantasy', 'Action', 'Foreign', 'Music',
'Documentary', 'Animation', 'Western', 'War'
Drama
Estas son las películas recomendadas en el género 'drama', ¡dale una oportunidad!



,original_title
0,The Million Dollar Hotel
1,Солярис
2,Once Were Warriors
3,Monsoon Wedding
4,Trois couleurs : Rouge


Análisis de usuario 123687

In [60]:
# Top 5 películas vistas por este usuario
top5_123687 = top_movies_by_user5(123687, top_n=5,df=df_ratings_peliculas)

top5_123687

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,movieId,original_title,rating,Generos
0,2288,Closer,5.0,"[Drama, Romance]"
1,1967,Ask the Dust,5.0,"[Drama, Romance]"
2,1372,Blood Diamond,4.5,"[Drama, Thriller, Action]"
3,849,Krull,4.5,"[Fantasy, Action, Adventure]"
4,441,Human Nature,4.5,"[Drama, Comedy]"


In [41]:
### Recomendaciones

recNLP123687, recSVD123687 = recomendacion_personalizada()

recNLP123687


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Por favor, introduce tu ID de usuario: 123687
ID de usuario ingresado: 123687

Usuario 123687. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores (NLP):


,IdPelicula,Título,Similitud Coseno,id,Generos
0,5123,August Rush,0.617161,5123,"[Family, Drama, Music]"
1,4369,Juste une question d'amour,0.606339,4369,"[Drama, Romance]"
2,235984,Goliyon Ki Raasleela Ram-Leela,0.593366,235984,"[Drama, Romance]"
3,6003,Romeo and Juliet,0.585540,6003,"[Drama, Romance]"
4,27904,แสงศตวรรษ,0.577350,27904,"[Drama, Romance]"


In [42]:
recSVD123687

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Indice,valor svd,Titulo,Genero
0,2528,1.164571,Madame Bovary,"[Drama, Foreign, Romance]"
1,2009,0.694973,"4 luni, 3 săptămîni și 2 zile",[Drama]
2,8914,0.585808,Deep Blue Sea,"[Action, Science Fiction, Thriller]"
3,1831,0.563071,The Next Best Thing,[Comedy]
4,8983,0.448371,Felidae,"[Animation, Mystery, Thriller]"


Análisis de usuario 6525




In [61]:
# Top 5 películas vistas por este usuario

top5_6525 = top_movies_by_user5(6525, top_n=5,df=df_ratings_peliculas)

top5_6525

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,movieId,original_title,rating,Generos
0,171,La Boum 2,5.0,[Comedy]
1,156,Wilbur begår selvmord,5.0,"[Comedy, Drama, Romance]"
2,262,The King of Comedy,5.0,"[Comedy, Drama]"
3,923,Dawn of the Dead,5.0,[Horror]
4,380,Rain Man,4.0,[Drama]


In [ ]:
recNLP6525, recSVD6525 = recomendacion_personalizada()
recNLP6525

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Por favor, introduce tu ID de usuario: 6525
ID de usuario ingresado: 6525

Usuario 6525. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores (NLP):


,Índice,Título,Similitud Coseno,id,Generos
0,122,La Boum,0.688062,122,"[Adventure, Fantasy, Action]"
1,26693,Moms' Night Out,0.565217,26693,[Horror]
2,20064,Diabolo menthe,0.531610,20064,"[Drama, Thriller]"
3,24962,Blueberry Hill,0.521286,24962,"[Adventure, Fantasy, Drama, Science Fiction, F..."
4,276,Harold and Maude,0.457330,276,"[Drama, Comedy]"
5,2049,Van Gogh,0.442807,2049,"[Documentary, Foreign]"
6,5876,Welcome Home Roscoe Jenkins,0.534522,5876,"[Science Fiction, Horror, Thriller]"


In [ ]:
recSVD6525

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Indice,valor svd,Titulo,Genero
0,175,1.133485,Le Grand Bleu,"[Adventure, Drama, Romance]"
1,28,0.950191,Apocalypse Now,"[Drama, War]"
2,290,0.862715,Barton Fink,"[Comedy, Drama]"
3,280,0.698489,Terminator 2: Judgment Day,"[Action, Thriller, Science Fiction]"
4,6440,0.667134,The Shipping News,"[Drama, Romance]"


Análisis de usuario 186200

In [62]:
# Top 5 películas vistas por este usuario

top5_186200 = top_movies_by_user5(186200, top_n=5,df=df_ratings_peliculas)

top5_186200


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,movieId,original_title,rating,Generos
0,1610,Fever Pitch,4.5,"[Comedy, Romance]"
1,780,La passion de Jeanne d'Arc,3.5,"[Drama, History]"
2,349,Crustacés et coquillages,3.0,[Comedy]
3,1690,Hostel,3.0,[Horror]
4,316,Halbe Treppe,3.0,"[Comedy, Drama]"


In [ ]:
recNLP186200, recSVD186200 = recomendacion_personalizada()

recNLP186200

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Por favor, introduce tu ID de usuario: 186200
ID de usuario ingresado: 186200

Usuario 186200. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores (NLP):


,Índice,Título,Similitud Coseno,id,Generos
0,9932,FC Venus,0.487950,9932,"[Comedy, Crime]"
1,12709,Unmade Beds,0.487950,12709,"[Comedy, Drama, Romance]"
2,14525,Orphans of the Storm,0.365148,14525,[Comedy]
3,14938,Joan of Arc,0.345033,14938,"[Romance, Comedy, Drama]"
4,275,Sommersturm,0.434613,275,"[Crime, Drama, Thriller]"
5,7345,After Sex,0.430331,7345,[Drama]
6,259,Mambo Italiano,0.389249,259,"[Comedy, Drama, Romance]"
7,11346,Zerophillia,0.353553,11346,[Comedy]


In [ ]:
recSVD186200

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Indice,valor svd,Titulo,Genero
0,53000,0.638138,Soul Assassin,"[Action, Mystery, Thriller]"
1,3635,0.528064,Girl with a Pearl Earring,"[Drama, Romance]"
2,3638,0.402483,No Reservations,[Comedy]
3,2989,0.331291,Roar,"[Adventure, Thriller]"
4,1882,0.317116,The Men,[Drama]


Análisis de usuario 270832

In [63]:

top5_270832 = top_movies_by_user5(270832, top_n=5,df=df_ratings_peliculas)

top5_270832

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,movieId,original_title,rating,Generos
0,165,Back to the Future Part II,5.0,"[Adventure, Comedy, Family, Science Fiction]"
1,292,Dave Chappelle's Block Party,5.0,"[Comedy, Documentary, Music]"
2,150,48 Hrs.,5.0,"[Thriller, Action, Comedy, Crime, Drama]"
3,317,Grbavica,5.0,"[Drama, History]"
4,597,Titanic,4.0,"[Drama, Romance, Thriller]"


In [ ]:
recNLP270832, recSVD270832 = recomendacion_personalizada()
recNLP270832

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Por favor, introduce tu ID de usuario: 270832
ID de usuario ingresado: 270832

Usuario 270832. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores (NLP):


,Índice,Título,Similitud Coseno,id,Generos
0,24827,Rubinrot,0.406181,24827,"[Drama, Comedy, Romance, Foreign]"
1,68,Back to the Future,0.389692,68,"[Comedy, Science Fiction]"
2,10055,Big Pun: The Legacy,0.478091,10055,"[Horror, Thriller]"
3,11355,Biggie and Tupac,0.478091,11355,"[Romance, Comedy, Drama]"
4,4972,Another 48 Hrs.,0.588348,4972,[Comedy]
5,26506,Convict,0.554700,26506,"[Drama, Romance]"


In [ ]:
recSVD270832

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Indice,valor svd,Titulo,Genero
0,227,0.180438,The Outsiders,"[Crime, Drama]"
1,193,0.104536,Star Trek: Generations,"[Science Fiction, Action, Adventure, Thriller]"
2,1792,0.092754,Stuck on You,[Comedy]
3,257,0.082665,Oliver Twist,"[Crime, Drama, Family]"
4,276,0.082034,Die fetten Jahre sind vorbei,"[Drama, Comedy]"
